In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F39829%2F61745%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T113222Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D085b7b4bb269e90114125b265bae771e4309135eac7c3176be6867ecdf3398c60a1d8ac991a6915a6a6b0fc9a90e9b3e9f47ffe54775d5a4826151d7e74638450d14eff5a1348abd062394c002aed3e7c6a2b553157448c543c026ac60fa96161c148bb6ee8a877ca144ea98f222281aaa180de4826f463572f9f71595b9249576df3a7a1fad451adddc5efa9afbd5a03d8a58494f3fc9a6d0b4b8ba803bb6dc7b5530e82e5f5af6fabb7e2d9b6e06737f3794d88371bb3b8e229b4f527d6daf93a5133c63ccddd2107f0389c8f25b5d4ba2b50f04201c131efb0f25a52d05d98a84992b4951263faa2bcfa52a5aa8bceea2d3516a9824242240d5c9e5ef6bf0'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


In [ ]:
---
title: "Electronic Products Pricing Report"
author: "Daniel Robles"
date: "May 17, 2019"
output: html_document
---

```{r setup, include=FALSE }
knitr::opts_chunk$set(echo = FALSE, warning = FALSE)
library(tidyverse)
library(forcats)
library(egg)
library(lubridate)
```

```{r originaldataload, include = FALSE}
data0 <- read_csv("../input/DatafinitiElectronicsProductsPricingData.csv") %>% select(-c(X27,X28,X29,X30,X31)) %>% rename(list.price = prices.isSale, store = prices.merchant, shipping.terms = prices.shipping, AZ.IDs = asins, ean.code = ean, upc.code = upc)
 # Void columns removed (created with the original file for line breaks)

data1 <- data0 %>% select(-prices.sourceURLs,-imageURLs, -keys, -ean.code, -sourceURLs) # Remove non-relevant data

data_new <-  data1 %>% filter(!(prices.condition %in% c("Seller refurbished", "refurbished", "Refurbished", "Manufacturer refurbished", "pre-owned", "Used")), !(manufacturer %in% c("5 Years", "2 Years", "120", "1-Year", "1 Year", "1 year warranty", "Manufacturers Limited Warranty:3 Year")))

data_new$prices.dateSeen <- as.Date(substr(data_new$prices.dateSeen, 0, 10))  # We take only the characters that define date and parse it properly

data_new <- filter(data_new,year(prices.dateSeen)>2016)

data_new <- data_new %>% mutate(month.dateSeen = month(prices.dateSeen, label = TRUE, abbr = TRUE)) %>% mutate(meanprice = ((prices.amountMax + prices.amountMin)/2))

# Recode alike manufacturers
data_new$manufacturer <- as.character(data_new$manufacturer)
data_new$manufacturer <- fct_recode(data_new$manufacturer,"Apple" = "Apple Computer", "Apple" = "Apple Computer (Direct)","Buffalo" = "Buffalo Technology","Dell" = "Dell Computer", "Dell" = "Dell Inc","Lenovo" = "Lenovo Corporate", "Lenovo" = "Lenovo Idea","Toshiba" = "TOSHIBA","Logitech" = "Logitech Inc","Samsung" = "SAMSUNG", "Samsung" = "Samsung IT", "Samsung" = "Samsung Electronics","SanDisk" = "Sandisk","Seagate" = "Seagate Technology")
#Remove the refurbished products sales

#-----------------------------
data_new$brand <- as.character(data_new$brand)
data_new$brand <- fct_recode(data_new$brand,"AfterShokz" = "Aftershokz","Asus" = "ASUS","AudioQuest" = "Audioquest","Belkin" = "Belkin Inc.","Bose" = "Bose®","Buffalo" = "Buffalo Technology","Cerwin-Vega" = "Cerwin Vega","Cobra" = "Cobra Electronics","Corsair" = "CORSAIR","Denaq" = "DENAQ","Denon" = "DENON - HEOS","Evga" = "EVGA","GoPro" = "Gopro","House of Marley" = "House Of Marley","Insignia" = "Insignia™","iSimple" = "Isimple","JBL" = "Jbl","JVC" = "Jvc","JVCM" = "Jvcm","Kicker" = "KICKER","Klipsch" = "Klipsch Xr8i In-Ear Headphones","LG" = "Lg","Master & Dynamic" = "Master Dynamic","MTX" = "MTX Audio","Netgear" = "NETGEAR","Panamax" = "PANAMX","Peerles" = "Peerles-AV","Pelican" = "PELICAN","PNY" = "Pny","Pro-Ject" = "Pro-Ject Audio Systems","Pyle" = "PYLE","Pyle" = "Pyle Pro","Pyle" = "PyleHome","Rode" = "RODE","SanDisk" = "Sandisk","Sanus" = "SANUS","Sol Republic" = "SOL REPUBLIC","StarTech" = "StarTech.com","TiVo" = "Tivo","TP-Link" = "Tp-Link","V-Moda" = "V-MODA","Vizio" = "VIZIO","Vizio" = "VIZIO, INC","Western Digital" = "WD")
#-----------------------------

data_new$prices.condition <- as.factor(data_new$prices.condition) #Convert this column to a factor
data_new$primaryCategories <- as.factor(data_new$primaryCategories) #Convert this column to a factor

data_new$shipping.terms <- as_factor(as.character(data_new$shipping.terms))
```

## About the dataset

The dataset used can be found at [Kaggle](https://www.kaggle.com/datafiniti/electronic-products-prices) and was created by Datafiniti. It contains several electronic products organized by ID and variables related to them: store, max price, min price, availability, etc.

First, we have make 3 initial considerations:

1. Brand new product sales; not used or refurbished product sales.
2. Major technology brands to compare among direct competitors.
3. Years 2017 and 2018 (2015 and 2016 have less data in comparison).

Our first approach is to narrow down the data to keep only the main IT brands:

## Two main branches

With the available data, we can observe at least two branches of market share for this analysis:

1. **Peripherals and accessories**: *Dell, HP Inc., Logitech, TRANSCEND, Samsung, SanDisk, Sony, Belkin, PNY*
2. **PC's and hardware**: *Acer, Apple, Asus, Dell, HP Inc, Lenovo, Samsung, Seagate Technology, Sony, Western Digital, Buffalo Technology, Toshiba.*

We will focus in brands associated within the second group

```{r filtered_businesses}
table <- data_new %>% select(-prices.condition)

list_peri <- c("Belkin", "Dell", "HP", "Logitech", "PNY", "Samsung", "Sony", "TRANSCEND")

list_pc <- c("Acer", "Apple", "Asus", "Buffalo", "Dell", "HP", "Lenovo", "Samsung", "Seagate", "Sony", "Western Digital", "Toshiba")

table_peri <- table %>%filter(brand %in% list_peri) %>% mutate(list.price = !list.price) %>% rename(Sale = list.price)

table_pc <- table %>% filter(brand %in% list_pc) %>% mutate(list.price = !list.price) %>% rename(Sale = list.price)
```

```{r plot_filter_brands, fig.align='center'}
table_peri$Sale <- as.character(table_peri$Sale)
table_peri$Sale <- fct_recode(table_peri$Sale, "Discount price"= "TRUE", "List price" = "FALSE")
# We don't discard the logical values for the general case

table_pc$Sale <- as.character(table_pc$Sale)
table_pc$Sale <- fct_recode(table_pc$Sale, "Discount price"= "TRUE", "List price" = "FALSE")
# We don't discard the logical values for the general case

plot_peri_flip <- table_peri %>% ggplot(aes(brand, fill = Sale)) + geom_bar(alpha = 0.3) + labs(x = "Brands", y = "", title = "Periph. & Acc. sales") + theme_minimal() + scale_fill_brewer(palette = "Set1") + coord_polar(clip = "off") + theme(legend.position = "none", aspect.ratio = 1) + ylim(-300,1600) #The scale is important, if it doesn't fit, the largest will appear blank
#This option prevent the margins from cutting the x axis labels
# theme(axis.text.x = element_text(angle = 55, hjust = 1))

plot_pc_flip <- table_pc %>% ggplot(aes(brand, fill = Sale)) + geom_bar(alpha = 0.3) + labs(x = "Brands", y = "", title = "PCs & Components sales") + theme_minimal() + scale_fill_brewer(palette = "Set1") + coord_polar(clip = "off") + theme(legend.position = "none", aspect.ratio = 1) + ylim(-300,1600)
#The scale is important, if it doesn't fit, the largest will appear blank
# theme(axis.text.x = element_text(angle = 55, hjust = 1))
#This option prevent the margins from cutting the x axis labels

grid.arrange(plot_peri_flip, plot_pc_flip, nrow = 1)
#egg library required
```

```{r filter_majorBrands_pc}
list_pc2 <- c("Acer", "Asus", "Buffalo", "Dell", "HP", "Lenovo", "Seagate", "Western Digital", "Toshiba")

table_pc2 <- table %>% filter(brand %in% list_pc2) %>% mutate(list.price = !list.price) %>% rename(Sale = list.price)
```

It can be observed that there are 3 brands that outscale our annalysis:

* *Sony* and *Samsung*: Offering not only sell PC components, but also *Home* and *Personal Accessories*, for instance.
* *Apple* has its own specific market share with specific technology and personal devices as *smartphones*.

In order to have our first analysis between brands, **we** will **exclude these 3 by now**. They can be treated in detail separately.

# PCs and Components brands

Let us compare the sales among brands, so we can have an idea of the competition (direct or indirect) in the following bar graphs. The second one shows the proportion of "discount/list price" sales by brand.

```{r plot_competitors_sales, message = FALSE, fig.align='center'}
# table_pc2$Sale <- as.character(table_pc2$Sale)
# table_pc2$Sale <- fct_recode(table_pc2$Sale, "Discount price"= "TRUE", "List price" = "FALSE")
#Cambiamos nombres para que aparezcan en la gráfica, pero en general, mantenemos la estructura lógica

plot_pc <- table_pc2 %>% ggplot(aes(brand, fill = Sale)) + geom_bar(alpha = 0.3) + labs(x = "Brands", y = "", title = "PCs & Components sales", caption = "This representation does not consider the three top 3 sellers") + theme_minimal() + theme(axis.text.x = element_text(angle = 55, hjust = 1)) + scale_fill_brewer(name = "Sale", labels = c("List price", "Discount price"), palette = "Set1")

plot_pc_prop <- table_pc2 %>% ggplot(aes(brand, fill = Sale)) + geom_bar(position = "fill", alpha = 0.3) + labs(x = "Brands", y = "", title = "Proportion of 'Discount/List price' in sales") + theme_minimal() + scale_y_continuous(labels = scales::percent,breaks = seq(0, 1, 0.2)) + theme(axis.text.x = element_text(angle = 55, hjust = 1)) + scale_fill_brewer(name = "Sale", labels = c("List price", "Discount price"), palette = "Set1")

ggarrange(plot_pc, plot_pc_prop)
#egg library required
```

# Monthly behavior

## Discount sales and revenue

We now analize the monthly behavior of sales and discounts (years 2017 and 2018). The facts we see in the following graph are:

1. Discounts occur more in the month of July, followed by August.
2. The **most sales without discount** occur during the months of **May and August**.
    * This is consistent observing the total revenue of this months.

```{r plot_monthly_discount, fig.align='center'}
plot_month1 <- table_pc2 %>% ggplot(aes(month.dateSeen, fill = Sale)) + geom_bar(position = "dodge", alpha = 0.3) + labs(x = "Months", y = "Sales", title = "Amount of sales in months") + theme_minimal() + theme(axis.text.x = element_text(angle = 55, hjust = 1)) + scale_fill_brewer(name = "Sale", labels = c("List price", "Discount price"), palette = "Set1")

table_pc2_top_sellers_store <- data_new %>% group_by(store) %>% filter(!is.na(store), n()>300) # Under this grouping, filter stores with more than 300 sales in the dataset

table_pc2_top_sellers_store$store <- as_factor(table_pc2_top_sellers_store$store)

plot_month2 <- table_pc2_top_sellers_store %>% ggplot(aes(month.dateSeen, meanprice, fill = month.dateSeen)) + geom_col(alpha = 0.4) + theme_minimal() + theme(axis.text.x = element_text(angle = 55, hjust = 1), legend.position = "none") + scale_fill_discrete(name = "Month") + labs(title = "Monthly electronics revenue", x = "Months", y = "Revenue")

ggarrange(plot_month1, plot_month2)
```

This high revenue may be explained by the "Back to School" season. So **it is convenient to have less discount prices for this kind of products.**

## Sales by store

If we see the amount of sales per month in a monthly basis, we can observe that "Best Buy" is the major vendor as it outscales our comparison among stores.
Nevertheless, there are other peaks in which another stores take part (BH Photovideo in January and December for instance).

It is a surprising fact that Amazon's sales level is way below its competitors. It seems that physical sales are preferred among customers when they buy electronics.

```{r plot_monthly_sales, fig.align='center'}
table_pc2_top_sellers_store %>% ggplot(aes(month.dateSeen, fill = store)) + geom_bar(position = "dodge", alpha = 0.3) + labs(x = "Months", y = "Sales") + theme_minimal()
```

```{r plot_monthly_revenue, include=FALSE, fig.align='center'}
table_pc2_top_sellers_store %>% filter(store != "Bestbuy.com") %>%
    ggplot(aes(month.dateSeen, meanprice, fill = month.dateSeen)) +
    geom_col(alpha = 0.5) +
    facet_wrap(~store) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 55, hjust = 1), legend.position = "none") +
    scale_fill_discrete(name = "Month") +
    labs(title = "Monthly revenue by store", x = "Months", y = "Revenue")

table_pc2_top_sellers_store %>% filter(store == "Bestbuy.com") %>%
    ggplot(aes(month.dateSeen, meanprice, fill = month.dateSeen)) +
    geom_col(alpha = 0.5) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 55, hjust = 1), legend.position = "none") +
    scale_fill_discrete(name = "Month") +
    labs(title = "Best Buy monthly revenue", x = "Months", y = "Revenue")
```

## Price distribution in sales

It is now interesting to see how the products are distributed according to their price among the stores analyzed.

As you can see from this diagram:

* *Beach Camera* have the most expensive purchased products, followed (by far) by *BH Photovideo*.
* The major distribution of the rest of the stores remain in the lower-price electronics.

```{r plot_stores_prices_distribution, fig.align='center'}
table_pc2_top_sellers_store %>% ggplot(aes(store, meanprice, color = store)) + geom_boxplot(aes(fill = store, alpha = 0.1)) + scale_y_continuous(limits = c(0,1500)) + coord_flip() + theme_minimal() + theme(axis.text.x = element_text(angle = 35, hjust = 1), legend.position = "none") + labs(title = "Prices distribution by store" , x = "Stores", y = "Prices")
```

You can observe the stores compared in the following grid (Best Buy is shown after because it outscales our graphics when grouped):

```{r plot_pricesfreq, message=FALSE, fig.align='center'}
table_pc2_top_sellers_store %>% filter(store != "Bestbuy.com") %>% ggplot(aes(meanprice, color = store)) + geom_freqpoly() + scale_x_continuous(limits = c(0,1500)) + labs(title = "Monthly distribution of prices compared by store", x = "Price", y = "Count") + facet_wrap(~month.dateSeen)

table_pc2_top_sellers_store %>% filter(store == "Bestbuy.com") %>% ggplot(aes(meanprice)) + geom_freqpoly() + scale_x_continuous(limits = c(0,1500)) + labs(title = "Monthly distribution of prices in Best Buy", x = "Price", y = "Count") + facet_wrap(~month.dateSeen)
```

As one can expect, the lower the product price, the major the sales are; independently of what store you observe.